In [ ]:
def extract_ghana_road_network(boundaries):
    """
    Extract road network from OpenStreetMap for Ghana.

    This function downloads a road network graph for Ghana, assigns speed limits 
    using OSM's 'maxspeed' attribute when available (otherwise estimates based on road type), 
    and computes travel times for each segment. The result is projected to UTM 
    and saved as a GraphML file for future use.

    Considerations:
    - Extracts all road types suitable for vehicle transport
    - Speed limits assigned based on OSM road classification
    - Network connectivity validated for routing algorithms
    
    Args:
        boundaries: GeoDataFrame with Ghana administrative boundaries
        
    Returns:
        networkx.MultiDiGraph: Road network graph with speed attributes
    """
    
    print("\nExtracting road network from OpenStreetMap...")
    
    # Get Ghana bounding box for OSM query
    ghana_bounds = boundaries.total_bounds  # [minx, miny, maxx, maxy]
    
    try:
        # Extract road network using bounding box
        # Network types: drive (all roads accessible by car)
        # Simplify: False to preserve all intersections for accurate routing
        print("Downloading road network from OSM (this may take 5-10 minutes)...")
        
        road_network = ox.graph_from_bbox(
            bbox=ghana_bounds,
            #north=ghana_bounds[3],   # max latitude
            #south=ghana_bounds[1],   # min latitude  
            #east=ghana_bounds[2],    # max longitude
            #west=ghana_bounds[0],    # min longitude
            network_type='drive',    # roads accessible by car
            simplify=False,          # preserve intersections
            retain_all=True         # keep disconnected components
        )
        
        print(f"✓ Downloaded network: {len(road_network.nodes)} nodes, {len(road_network.edges)} edges")

        # Define fallback speed limits (in km/h) for each road type if 'maxspeed' is missing
        # Add speed attributes based on OSM highway classification
        # Speed assignment follows OSM tagging conventions for Ghana
        speed_mapping = {
            'motorway': 80,      # Major highways
            'trunk': 70,         # National roads
            'primary': 60,       # Regional roads  
            'secondary': 50,     # District roads
            'tertiary': 40,      # Local roads
            'unclassified': 30,  # Minor roads
            'residential': 25,   # Urban residential
            'track': 20,         # Rural tracks
            'path': 15          # Walking paths accessible by motorbike
        }

        # Counter to keep track of how many edges had actual maxspeed data
        used_maxspeed_count = 0

        
        # Apply speeds to network edges through a Loop through each road segment (edge) in the network
        for u, v, key, data in road_network.edges(data=True, keys=True):
            highway_type = data.get('highway', 'unclassified')   # Get the 'highway' tag from OSM (e.g., 'primary', 'residential')
            
            # Handle highway type lists (some edges have multiple types)
            if isinstance(highway_type, list):
                highway_type = highway_type[0]  # Sometimes 'highway' is a list (e.g., ['residential', 'service']), take the first
                
            # Assign speed based on highway type
            #speed_kmh = speed_mapping.get(highway_type, 30)  # default 30 km/h

            # OSM 'maxspeed' if available
            maxspeed = data.get('maxspeed')
            speed_kmh = None  # we'll determine this shortly

            #Try to get the actual 'maxspeed' from OSM data
            if maxspeed:
                 # Handle various formats: e.g., '50', '50 km/h', ['50', '60']
                 if isinstance(maxspeed, list):
                     maxspeed = maxspeed[0] # If maxspeed is a list (e.g., ['50', '60']), take the first

                 try:
                     speed_kmh = int(str(maxspeed).split()[0])  # Extract numeric value from '50 km/h' or '50'
                     used_maxspeed_count += 1  # track usage
                 except:
                     speed_kmh = speed_mapping.get(highway_type, 30)  # Fall back to default speed if parsing fails
            else:
                 speed_kmh = speed_mapping.get(highway_type, 30)  # If 'maxspeed' not available, use speed based on road type


            # Store speed in km/h for travel time calculations
            road_network[u][v][key]['speed_kmh'] = speed_kmh  # storing the estimated or actual speed on the edge 

            # Calculate travel time in hours based on length and speed
            # Estimate travel time (in hours) = length / speed
            length_km = data.get('length', 100) / 1000  # convert m to km and default to 100m if missing
            travel_time_hours = length_km / speed_kmh
            road_network[u][v][key]['travel_time'] = travel_time_hours
        
        print(f"✓ Assigned speeds to {len(road_network.edges)} road segments")
        rint(f"✓ Used actual OSM 'maxspeed' on {used_maxspeed_count:,} edges")
        
        # Project to a coordinate system for Ghana (UTM Zone 30N)
        road_network = ox.project_graph(road_network, to_crs='EPSG:32630')
        print("✓ Projected network to UTM Zone 30N")

        # Save the processed road network to file 
        ox.save_graphml(road_network, filepath='ghana_road_network.graphml')
        print("✓ Saved road network to 'ghana_road_network.graphml'")
        
        return road_network
        
    except Exception as e:
        print(f"✗ Error extracting road network: {e}")  # Error handling if download or processing fails
        print("This may be due to network connectivity or OSM server issues")
        print("Consider using a cached network or smaller bounding box")
        return None

# Extract road network from OpenStreetMap
if local_data:
    road_network = extract_ghana_road_network(local_data['boundaries'])
    print(f"\nPhase 1 Complete: All datasets loaded successfully")
    print(f"- Road network: {len(road_network.nodes):,} intersections")
    print(f"- Hospital facilities: {len(local_data['hospitals'])}")
    print(f"- Analysis extent: Ghana national boundaries")
else:
    print("✗ Cannot proceed without local datasets")